# Run the tracker

This jupyter notebook bundles all necessary high-level steps to train the tracker. We start with running the simulation to generate training data.

In [1]:
from tqdm import tqdm
import pickle
import sys
import numpy as np
from datetime import datetime
import os
sys.path.append('build/')
import simulation
from time import time, sleep
from python.plotter import Plotter
from python.GNNtracker import GNNTracker
from python.constants import *

Now we need to get simulation data of the warehouse. We can use the simulation to generate new simulation data. This cell is optional, we can also load old simulation data.

In [6]:
T_simulation = 60 * 60 * 1 # 1 hour

sim = simulation.Simulation(
    T_step=0.1, 
    N_humans=3, 
    N_robots=1
    )
sim_states = []
N_minutes = int(T_simulation / 60)
N_hours = int(T_simulation / 3600)
pbar = tqdm(range(0, N_minutes), desc='Simulation')
for i in pbar:
    sim_states += sim.step(int(60 / sim.T_step))
    current_hour = int(i / 60)
    current_minute = i % 60
    pbar.set_postfix({'Simulated time': f'{current_hour}:{current_minute} of {N_hours} hours'})

filename = os.path.join(LOG_FOLDER, 'log_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.pkl')
with open(filename, 'wb') as outp:
    pickle.dump({
        'T_step': sim.T_step,
        'T_simulation': np.arange(0, len(sim_states) * sim.T_step, sim.T_step),
        'N_humans': sim.N_humans,
        'N_robots': sim.N_robots,
        'sim_states': sim_states,
    }, outp, pickle.HIGHEST_PROTOCOL)
    

Simulation: 100%|██████████| 60/60 [00:02<00:00, 24.37it/s, Simulated time=0:59 of 1 hours]


Now we load the generated simulation logs.

In [2]:
if "filename" not in locals():
    filename = os.path.join(LOG_FOLDER, 'log_2024-08-16_20-34-51.pkl')
with open(filename, 'rb') as f:
    sim_log = pickle.load(f)
sim_states = sim_log['sim_states']
T_simulation = sim_log['T_simulation']
T_step = sim_log['T_step']
N_humans = sim_log['N_humans']
N_robots = sim_log['N_robots']

Now that we have the simulation data, we can run the actual tracker.

In [3]:
plot = True
record_video = False
realtime = True

tracker = GNNTracker(
    T_step=T_step,
    train=False,
)

if plot: plotter = Plotter(record_frames=record_video)

for sim_state in sim_states:
    start = time()
    
    tracker.add_observation(sim_state)
    node_probabilities = tracker.predict()

    if plot: plotter.update(sim_state, node_probabilities)    
    
    if realtime: sleep(max(0, T_step - (time() - start)))

if record_video: plotter.create_video(T_step)

KeyboardInterrupt: 

Now that the simulation has been run and the logs have been saved, we can load the loads and extract the data needed for training the Graph Convolutional Net in the tracker. The data is stored in such way that it can be loaded by pytorch-geometric.

In [24]:
import pickle
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from python.tracker import Tracker
import torch
import numpy as np

data_list = [Data(...), ..., Data(...)]
loader = DataLoader(data_list, batch_size=32)


with open(filename, 'rb') as f:
    sim_log = pickle.load(f)
sim_state = sim_log['sim_state']


from python.constants import GRAPH_PATH
import json

# load graph
with open(GRAPH_PATH, 'rb') as f:
    graph_data = json.load(f)
nodes = graph_data['nodes']
edges = graph_data['edges']

robot_observations = []
for i in range(len(sim_state)-1):
    probabilities, confidences = Tracker.extract_observation_from_state(sim_state[0])
    next_probabilities, next_confidences = Tracker.extract_observation_from_state(sim_state[1])

    robot_observations.append(Data(
        # x=torch.hstack((torch.tensor(probabilities[:,np.newaxis]), torch.tensor(confidences[:,np.newaxis]))), 
        x=torch.tensor(probabilities[:,np.newaxis]),
        y=torch.tensor(next_probabilities[:,np.newaxis]),
        edge_index=torch.tensor(edges).T)
        )

loader = DataLoader(robot_observations, batch_size=128)

Now that we have a DataLoader for our data, we can define our network architecture.

In [25]:
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        # Define your layers here
        self.conv1 = GCNConv(1, 8)
        self.conv2 = GCNConv(8, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
          
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        
        return x

Now we train the network.

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)

model.train()
for epoch in range(20):
    total_loss = 0
    for data in loader:
        data = data.to(device)
        data.x = data.x.float()
        optimizer.zero_grad()
        out = model(data)
        loss = F.binary_cross_entropy_with_logits(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss/len(loader)}')

Epoch 1, Loss: 0.47653289093650963
Epoch 2, Loss: 0.2267185828429523
Epoch 3, Loss: 0.16100687461671984
Epoch 4, Loss: 0.13177482895166961
Epoch 5, Loss: 0.09415652318789017
Epoch 6, Loss: 0.06902463443930916
Epoch 7, Loss: 0.06070562598813252
Epoch 8, Loss: 0.05616342912459074
Epoch 9, Loss: 0.05224546466573497
Epoch 10, Loss: 0.043209311606710994
Epoch 11, Loss: 0.02904659745410954
Epoch 12, Loss: 0.022102839624904502
Epoch 13, Loss: 0.017662046813551883
Epoch 14, Loss: 0.01403605539617939
Epoch 15, Loss: 0.012278992077056263
Epoch 16, Loss: 0.011746225032124174
Epoch 17, Loss: 0.011566076423633437
Epoch 18, Loss: 0.011496834899972056
Epoch 19, Loss: 0.011461851064601495
Epoch 20, Loss: 0.011440883968299814


Now the model is trained and can be saved.

In [30]:
torch.save(model, 'models/gcn_model_full.pth')
print('Entire model saved to gcn_model_full.pth')

Entire model saved to gcn_model_full.pth
